# Race Condition Vulnerabilities


### Three Examples 
#### Example 1
The following PHP code introduces the example of a race condition: in banking system, when two withdraw of \$90 requests take place in a short interval, instead of accepting one and rejecting the other, the system could potentially withdraw \$180, which is undefined behavior. 

```php
function withdraw($amount)
{
    $balance = getBalance();
    if ($amount <= $balance) {
        $balance = $balance - $amount;
        echo "You have withdrawn: $amount";
        saveBalance($balance);
    }
    else {
        echo "Insufficient Fund."; 
    }
}
```

__Time of Check Time of Use__: A special type of race condition in software that occurs when checking a condition before using a resource. The condition could change between time of use and time of check. <br>
__Dirty COW__: allows attacker to modify any protected file, as only as file is readable to the attacker. It is able to be exploited for privilege escalation. Affects Linux and Android.

#### Example 2
The property of a SET-UID program is that when it is executed by a normal user, its _effective UID_ is root, but real UID is not root. The following program is SET-UID program and wants to write to /tmp, which is globally writable. The SET-UID program has root privilege to write to any files, but to enforce that only the real user can write to the file, the program checks if the real UID is correct. This is done through `access()` call. The program thus checks if the real user has the privilege to write to the file `/tmp/x`. 
```c
if (!access("/tmp/x", W_OK)) 
{
    f = open("/tmp/x", O_WRITE);
    write_to_file(f); 
}
else 
{
    fprintf(stderr, "Permission denied\n");
}
```
`open()` system call checks the effective UID of the program, whereas `access()` checks the real user ID for the program. The code on the system call `open()` from execution. However, there is a window between the time when file is checked and time when file is opened. <br>
To make use of the vulnerability in this 'blink of time', we can think of using __symbolic link__. Suppose that we want to modify the file `/etc/passwd`, but we can only write to `/tmp/x`. Now: 
* Before running the privileged program, we create a regular file X inside the `/tmp` directory. 
* This will pass `access()` check since it is our own file. 
* Right after `access()` can before program reaches `open()`, we quickly change `/tmp/x` to a symbolic link to the `/etc/passwd`. 
* When `open()` is invoked, it will open the passwd file, and since it only checks euid, now the program has write access to the file.
Since the modern processor runs instructions in less than miliseconds, it is very unlikely to successfully execute the linking in between. However, with large number of tries, it is possible to do the work right in the interval. 

To actually perform an attack, we need:
* The vulnerable program to run in a loop
* Run the attack program at the same time.
The __attacker__ runs the following instructions relentlessly: 
* __A1__: Make the `/tmp/x` point to a file owned by us. (simply create this file)
* __A2__: Make `/tmp/x` point to `passwd` (by symbolic linking) 
The __vulnerable__ program runs the following instructions relentlessly:
* __V1__: check users permission on `/tmp/x`
* __V2__: open the `/tmp/x` file for write

As long as the final sequence of execution is: $ \textbf{A}_1 \rightarrow \textbf{V}_1 \rightarrow \textbf{A}_2\rightarrow \textbf{V}_2$, we can achieve the goal.

#### Example 3
In the following program, another SET-UID, examines if a file exists, if not, it will create it and write to it. 
```c
file = "/tmp/x";
fileExist = check_file_existance(file);
if (fileExist == FALSE) 
{
    f = open(file, O_CREAT);
    //write 
    ...
}
```
So if between the check existance and the `open()`, we somehow make the name a symbolic link to `\etc\passwd`, we can again have write access to the passwd file. 

### Experiment Set up 
The following vulnerable program has the same problem as the `access` and `open` problem introduced before <br>
```c
#include <stdio.h>
#include <unistd.h>

int main() 
{
    char *fn = "/tmp/XYZ";
    char buffer[60];
    FILE *fp;
    
    scanf("%50s", buffer);
    if (!access(fn, W_OK)) 
    {
        fp = fopen(fn, "a+");
        fwrite("\n", sizeof(char), 1, fp);
        fwrite(buffer, sizeof(char), strlen(buffer), fp); 
        
    }
                           
    
}
```
Set it as SET-UID, as:
```
gcc vulp.c -o vulp
sudo chown root vulp
sudo chmod 4755 vulp
```
Then we need to turn off the Ubuntu countermeasure that does not allow symbolic link to a world-writable directory. 
```
sudo sysctl -w kernel.yama.protected_sticky_symlinks=0 
```